In [ ]:
!pip install SPARQLWrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
!conda install SPARQLWrapper

Channels:
 - defaults
 - conda-forge
 - stanfordnlp
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



# Import Libraries

In [ ]:
import gc

import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import os
import re

import itertools
import random
random.seed(8100)

import numpy as np
np.random.seed(0)

# Taken from handi's github

## Update blazegraph

In [ ]:
import os
import requests
import time

# Blazegraph server configuration
# endpoint_url = 'http://localhost:8100/blazegraph/sparql'
endpoint_url = 'http://localhost:7200/repositories/lexid'
# update_url = 'http://localhost:8100/blazegraph/namespace/kb/sparql'
update_url = 'http://localhost:7200/repositories/lexid/statements'
# headers = {'Content-type': 'application/x-turtle'}
headers = {
    'Content-type': 'application/x-turtle',
    'Accept': 'application/json'
}

# endpoint_url = 'http://34.143.173.88:9999/blazegraph/'
# update_url = 'http://34.143.173.88:9999/blazegraph/namespace/kb/sparql'
# headers = {'Content-type': 'application/x-turtle'}

def upload(path):
  with open(path, "rb") as f:
    data = f.read()


locations = []
list_folder = ['lexid/data/bn', 'lexid/data/lain-lain', 'lexid/data/ln', 'lexid/data/perda']
# list_folder = ['lexid/data/bn']
for folder in list_folder:
    for (root, dirs, files) in os.walk(folder, topdown=True):
        for arr in files:
            if root[-1:] != "\\":
                locations.append("{}\{}".format(root, arr))
            else:
                locations.append("{}{}".format(root, arr))

start_time = time.time()
# send turtle files to server
file_executed = 0
failed_document = []
data_to_process = len(locations)
for file_name in locations:
    file_executed += 1
    # if data_to_process < 25:
    #     with open(file_name, 'rb') as f:
    #         data = f.read()
    #         response = requests.post(update_url, headers=headers, data=data)
    #         if response.status_code == 200:
    #             print('Data from file {} is successfully uploaded'.format(file_name))
    #         else:
    #             print('Failed to upload data from file {}'.format(file_name))
    #             failed_document.append(file_name)
    with open(file_name, 'rb') as f:
        data = f.read()
        data_decoded = data.decode("utf-8")
        # Change '/ ' to '/'
        data_decoded = re.sub(r"/\s", r"/", data_decoded)
        data = data_decoded.encode("utf-8")
        response = requests.post(update_url, headers=headers, data=data)
        if response.status_code == 200 or response.status_code == 204:
            print('Data from file {} is successfully uploaded'.format(file_name))
        else:
            print('Failed to upload data from file {}'.format(file_name))
            failed_document.append(file_name)
    data_to_process -= 1
    print(f"{data_to_process} files remaining")
end_time = time.time()
print("Processing Time: " + str(end_time - start_time))

# failed data
print("======================FAILED DOCUMENT============================")
print(len(failed_document))
for i in failed_document:
    print(i)

with open("failed-to-upload.txt", 'w') as f:
    for i in failed_document:
        f.write(i)

Streaming output truncated to the last 5000 lines.
2498 files remaining
Data from file lexid/data/bn\2010\bn444-2010.ttl is successfully uploaded
2497 files remaining
Data from file lexid/data/bn\2010\bn452-2010.ttl is successfully uploaded
2496 files remaining
Data from file lexid/data/bn\2010\bn458-2010.ttl is successfully uploaded
2495 files remaining
Data from file lexid/data/bn\2010\bn46-2010.ttl is successfully uploaded
2494 files remaining
Data from file lexid/data/bn\2010\bn483-2010.ttl is successfully uploaded
2493 files remaining
Data from file lexid/data/bn\2010\bn517-2010.ttl is successfully uploaded
2492 files remaining
Data from file lexid/data/bn\2010\bn550-2010.ttl is successfully uploaded
2491 files remaining
Data from file lexid/data/bn\2010\bn563-2010.ttl is successfully uploaded
2490 files remaining
Data from file lexid/data/bn\2010\bn570-2010.ttl is successfully uploaded
2489 files remaining
Data from file lexid/data/bn\2010\bn575-2010.ttl is successfully uploaded


# Question Generation

## Preparation

### SPARQL Wrapper

In [ ]:
# sparql = SPARQLWrapper(os.environ.get('URL', 'http://localhost:8100/blazegraph/sparql'))
sparql = SPARQLWrapper(os.environ.get('URL', 'http://localhost:7302/repositories/lexid'))
sparql.setReturnFormat(JSON)
# locale.setlocale(locale.LC_ALL, "id_ID.UTF-8")
prefix = '''
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX wd: <https://www.wikidata.org/wiki/>
PREFIX lexid-s: <https://w3id.org/lex-id/schema/>
PREFIX lexid: <https://w3id.org/lex-id/data/>
'''

### Generation Class

In [ ]:
class linked_generation:
  def __init__(self, words, p=np.array([]), rng=None):
    if rng is None:
      self.rng = np.random.default_rng(12345)
    else:
      self.rng = rng
    self.words = words
    self.p = p
    if self.p.size == 0:
      self.types = 'single'
    else:
      self.types = 'multiple'
    self.next = None

  def set_next(self, next):
    self.next = next

  def random_sequence(self):
    if self.types == 'single':
      sequence = ''
      if (self.words.size != 0):
        sequence = self.rng.choice(self.words)

      if (self.next is None):
        return sequence
      else:
        next_sequence = self.next.random_sequence()
        if (sequence == '' or next_sequence == ''):
          return sequence + next_sequence
        else:
          return sequence + ' ' + next_sequence
    elif self.types == 'multiple':
      linked_sequence = self.rng.choice(self.words, p=self.p)

      if (self.next is None):
        return linked_sequence.random_sequence()
      else:
        sequence = linked_sequence.random_sequence()
        next_sequence = self.next.random_sequence()

        if (sequence == '' or next_sequence == ''):
          return sequence + next_sequence
        else:
          return sequence + ' ' + next_sequence

Link Query SPARQL: [https://github.com/Keyza-asyadda/query-lexidv2](https://github.com/Keyza-asyadda/query-lexidv2)

## Question QS

#### Perlu bikin versi non-baku-nya juga gak?
contoh: "Apakah legal_title masih dipraktikkan?" jadi "Apakah legal_title masih dipraktekkan?"

### TS : Semantic Question

#### TS Images for combination

![T1 Active](https://drive.google.com/uc?export=view&id=1oueGQ1JVU7nWwYNQGtHVcu_sCiWhJVow)

![T1 Passive](https://drive.google.com/uc?export=view&id=1jXpPbQwfOokXsVt_tJgbtkj4MaODlxBJ)

![T1 Noun V1](https://drive.google.com/uc?export=view&id=1JQajLSfWQWzqgYQm5NfYHP9_5qA0rfcZ)

![T1 Noun V2](https://drive.google.com/uc?export=view&id=1_JIJedsX05Ve18RevdGDj79WkavlE7IO)

#### TS Code

In [ ]:
print('Generating Question for TS : Semantic Questions')

class ts_qc:
  def __init__(self, seed=12345):
    self.rng = np.random.default_rng(seed)

    self.asked_modality_entity = np.array(['subject', 'modality', 'object', 'qualifier', 'none'])
    self.asked_entity = np.array(['subject', 'object', 'qualifier', 'none'])

    self.qualifier_position = np.array(['front', 'back'])

  def run(self, query_result):
    list_of_answers = []

    list_of_data = query_result.copy()
    data_to_process = len(list_of_data)

    index_loop= 0
    while data_to_process > 0:
      # Ignore 'Concept ' in front of words
      subject_word = list_of_data[index_loop].get('subject_label')[8:]
      modality_word = list_of_data[index_loop].get('modality_label')[8:]
      act_word = list_of_data[index_loop].get('act_label')[8:]
      object_word = list_of_data[index_loop].get('object_label')[8:]
      qualifier_type_word = list_of_data[index_loop].get('qualifier_type_label')[8:]
      qualifier_value_word = list_of_data[index_loop].get('qualifier_value_label')[8:]

      asked_selected = ''
      if modality_word == '':
        asked_selected = self.rng.choice(self.asked_entity)
      else:
        asked_selected = self.rng.choice(self.asked_modality_entity)

      qualifier_selected = self.rng.choice(self.qualifier_position)

      generated_question = ''
      ner = []
      solution = ''
      complete_sentence = subject_word
      if modality_word != '':
        complete_sentence += ' ' + modality_word
      complete_sentence += ' ' + act_word
      complete_sentence += ' ' + object_word
      complete_sentence += ' ' + qualifier_type_word
      complete_sentence += ' ' + qualifier_value_word

      if (asked_selected == 'subject'):
        if qualifier_selected == 'front':
          generated_question += qualifier_type_word
          ner.extend(['B-QTY' if i == 0 else 'I-QTY' for i in range(len(qualifier_type_word.split()))])
          generated_question += ' ' + qualifier_value_word + ', '
          ner.extend(['B-QVA' if i == 0 else 'I-QVA' for i in range(len(qualifier_value_word.split()))])
        generated_question += 'apa yang'
        ner.extend(['O', 'O'])
        if modality_word != '':
          generated_question += ' ' + modality_word
          ner.extend(['B-MDL' if i == 0 else 'I-MDL' for i in range(len(modality_word.split()))])
        generated_question += ' ' + act_word
        ner.extend(['B-ACT' if i == 0 else 'I-ACT' for i in range(len(act_word.split()))])
        generated_question += ' ' + object_word
        ner.extend(['B-OBJ' if i == 0 else 'I-OBJ' for i in range(len(object_word.split()))])
        if qualifier_selected == 'back':
          generated_question += ' ' + qualifier_type_word
          ner.extend(['B-QTY' if i == 0 else 'I-QTY' for i in range(len(qualifier_type_word.split()))])
          generated_question += ' ' + qualifier_value_word
          ner.extend(['B-QVA' if i == 0 else 'I-QVA' for i in range(len(qualifier_value_word.split()))])
        solution = subject_word
      elif (asked_selected == 'modality'):
        if qualifier_selected == 'front':
          generated_question += qualifier_type_word
          ner.extend(['B-QTY' if i == 0 else 'I-QTY' for i in range(len(qualifier_type_word.split()))])
          generated_question += ' ' + qualifier_value_word + ', '
          ner.extend(['B-QVA' if i == 0 else 'I-QVA' for i in range(len(qualifier_value_word.split()))])
        generated_question += modality_word + 'kah'
        ner.extend(['B-MDL' if i == 0 else 'I-MDL' for i in range(len(modality_word.split()))])
        generated_question += ' ' + subject_word
        ner.extend(['B-SBJ' if i == 0 else 'I-SBJ' for i in range(len(subject_word.split()))])
        generated_question += ' ' + act_word
        ner.extend(['B-ACT' if i == 0 else 'I-ACT' for i in range(len(act_word.split()))])
        generated_question += ' ' + object_word
        ner.extend(['B-OBJ' if i == 0 else 'I-OBJ' for i in range(len(object_word.split()))])
        if qualifier_selected == 'back':
          generated_question += ' ' + qualifier_type_word
          ner.extend(['B-QTY' if i == 0 else 'I-QTY' for i in range(len(qualifier_type_word.split()))])
          generated_question += ' ' + qualifier_value_word
          ner.extend(['B-QVA' if i == 0 else 'I-QVA' for i in range(len(qualifier_value_word.split()))])
        solution = 'ya'
      elif (asked_selected == 'object'):
        if qualifier_selected == 'front':
          generated_question += qualifier_type_word
          ner.extend(['B-QTY' if i == 0 else 'I-QTY' for i in range(len(qualifier_type_word.split()))])
          generated_question += ' ' + qualifier_value_word + ', '
          ner.extend(['B-QVA' if i == 0 else 'I-QVA' for i in range(len(qualifier_value_word.split()))])
        generated_question += 'apa yang'
        ner.extend(['O', 'O'])
        if modality_word != '':
          generated_question += ' ' + modality_word
          ner.extend(['B-MDL' if i == 0 else 'I-MDL' for i in range(len(modality_word.split()))])
        generated_question += ' ' + subject_word
        ner.extend(['B-SBJ' if i == 0 else 'I-SBJ' for i in range(len(subject_word.split()))])
        generated_question += ' ' + act_word
        ner.extend(['B-ACT' if i == 0 else 'I-ACT' for i in range(len(act_word.split()))])
        if qualifier_selected == 'back':
          generated_question += ' ' + qualifier_type_word
          ner.extend(['B-QTY' if i == 0 else 'I-QTY' for i in range(len(qualifier_type_word.split()))])
          generated_question += ' ' + qualifier_value_word
          ner.extend(['B-QVA' if i == 0 else 'I-QVA' for i in range(len(qualifier_value_word.split()))])
        solution = object_word
      elif (asked_selected == 'qualifier'):
        generated_question += qualifier_type_word + ' apa'
        ner.extend(['B-QTY' if i == 0 else 'I-QTY' for i in range(len(qualifier_type_word.split()))])
        ner.append('O')
        generated_question += ' ' + subject_word
        ner.extend(['B-SBJ' if i == 0 else 'I-SBJ' for i in range(len(subject_word.split()))])
        if modality_word != '':
          generated_question += ' ' + modality_word
          ner.extend(['B-MDL' if i == 0 else 'I-MDL' for i in range(len(modality_word.split()))])
        generated_question += ' ' + act_word
        ner.extend(['B-ACT' if i == 0 else 'I-ACT' for i in range(len(act_word.split()))])
        generated_question += ' ' + object_word
        ner.extend(['B-OBJ' if i == 0 else 'I-OBJ' for i in range(len(object_word.split()))])
        solution = qualifier_type_word + ' ' + qualifier_value_word
      elif (asked_selected == 'none'):
        if qualifier_selected == 'front':
          generated_question += qualifier_type_word
          ner.extend(['B-QTY' if i == 0 else 'I-QTY' for i in range(len(qualifier_type_word.split()))])
          generated_question += ' ' + qualifier_value_word + ', '
          ner.extend(['B-QVA' if i == 0 else 'I-QVA' for i in range(len(qualifier_value_word.split()))])
        generated_question += 'apakah'
        ner.append('O')
        generated_question += ' ' + subject_word
        ner.extend(['B-SBJ' if i == 0 else 'I-SBJ' for i in range(len(subject_word.split()))])
        if modality_word != '':
          generated_question += ' ' + modality_word
          ner.extend(['B-MDL' if i == 0 else 'I-MDL' for i in range(len(modality_word.split()))])
        generated_question += ' ' + act_word
        ner.extend(['B-ACT' if i == 0 else 'I-ACT' for i in range(len(act_word.split()))])
        generated_question += ' ' + object_word
        ner.extend(['B-OBJ' if i == 0 else 'I-OBJ' for i in range(len(object_word.split()))])
        if qualifier_selected == 'back':
          generated_question += ' ' + qualifier_type_word
          ner.extend(['B-QTY' if i == 0 else 'I-QTY' for i in range(len(qualifier_type_word.split()))])
          generated_question += ' ' + qualifier_value_word
          ner.extend(['B-QVA' if i == 0 else 'I-QVA' for i in range(len(qualifier_value_word.split()))])
        solution = 'ya'


      generated_question = re.sub(r'^(\w)', lambda first_char: first_char.group(1).upper(), generated_question)
      solution = re.sub(r'^(\w)', lambda first_char: first_char.group(1).upper(), solution)

      row = [
         generated_question,
         solution,
         ner,
         complete_sentence
      ]
      list_of_answers.append(row)
      data_to_process -= 1
      index_loop += 1

    return np.array(list_of_answers)

Generating Question for TS : Semantic Questions


#### TS Get all information

In [ ]:
ts_query = '''
SELECT DISTINCT ?Rule ?SubjectLabel ?ModalityLabel ?ActTypeLabel ?ObjectLabel ?QualifierTypeLabel ?QualifierValueLabel
WHERE {
    {
        SELECT distinct ?Rule ?Act
        WHERE {
            ?Rule a lexid-s:Norm ;
                lexid-s:hasAct ?Act .
            ?Act lexid-s:hasQualifier ?Qualifier ;
            	rdfs:label ?ActLabel .

            FILTER(REGEX(?ActLabel, "^Act 1", "i"))
        }
        GROUP BY ?Rule ?Act
        HAVING (COUNT(*) = 1)
	}
    ?Act lexid-s:hasSubject ?SubjectContent ;
    	lexid-s:hasObject ?Object ;
    	rdfs:label ?ActLabel ;
    	lexid-s:hasActType ?ActType .
    ?SubjectContent a lexid-s:LegalDocumentContent ;
    	rdfs:label ?SubjectContentLabel ;
        lexid-s:hasSubject/rdfs:label ?SubjectLabel .
    ?Object rdfs:label ?ObjectLabel .
    ?ActType rdfs:label ?ActTypeLabel .
    OPTIONAL {
        ?Act lexid-s:hasModality ?Modality .
        ?Modality rdfs:label ?ModalityLabel
    }

    ?Act lexid-s:hasQualifier ?Qualifier .
    ?Qualifier a lexid-s:LegalDocumentContent ;
    	rdfs:label ?QualifierContentLabel ;
    	lexid-s:hasQualifierType/rdfs:label ?QualifierTypeLabel ;
    	lexid-s:hasQualifierValue ?QualifierValue .
    ?QualifierValue rdfs:label ?QualifierValueLabel .

    FILTER(REGEX(?SubjectContentLabel, "^Concept 1", "i")
        && !REGEX(?SubjectLabel, "^Concept \\\\d", "i")
        && REGEX(?SubjectLabel, "^Concept", "i")
        && !REGEX(?ObjectLabel, "^Concept \\\\d", "i")
        && REGEX(?ObjectLabel, "^Concept", "i")
        && REGEX(?ActLabel, "^Act 1", "i")
        && REGEX(?ActTypeLabel, "^Concept", "i")
        && REGEX(?ModalityLabel, "^Concept", "i")
        && REGEX(?QualifierTypeLabel, "^Concept", "i")
        && ?QualifierTypeLabel NOT IN ("Concept Dep", "Concept Undefined", "Concept Advmod", "Concept Xcomp", "Concept Obl", "Concept Acl", "Concept Det", "Concept Compound Plur", "Concept Fixed", "Concept Nummod", "Concept Mark", "Concept Case", "Concept Nmod")
        && REGEX(?QualifierValueLabel, "^Concept", "i")
        && REGEX(?QualifierContentLabel, "^Concept 2", "i"))
}
'''

# def t1_query_dataset(legal_document):
#   return '''
#   SELECT DISTINCT ?answer
#   WHERE {
#       <''' + legal_document + '''> lexid-s:hasEnactionOfficial ?Official .
#       ?Official rdfs:label ?answer .
#   }
#   '''

sparql.setQuery(prefix + ts_query)

ts_informations_arr = []
try:
    result = sparql.queryAndConvert()
    for ans in result['results']['bindings']:
      chunk = {
          'rule' : ans['Rule']['value'],
          'subject_label' : ans['SubjectLabel']['value'],
          'modality_label' : ans['ModalityLabel']['value'],
          'act_label' : ans['ActTypeLabel']['value'],
          'object_label' : ans['ObjectLabel']['value'],
          'qualifier_type_label' : ans['QualifierTypeLabel']['value'],
          'qualifier_value_label' : ans['QualifierValueLabel']['value'],
      }
      ts_informations_arr.append(chunk)
    # if not answer:
    #     return "Jawaban tidak ditemukan"
    # return answer if question_type not in [2, 3] else datetime.strptime(answer, "%Y-%m-%d").strftime("%d %B %Y")

except Exception as e: print(e)

ts_informations = np.array(ts_informations_arr)
print(len(ts_informations))

802


In [ ]:
# pd.DataFrame(data=[[d['legal_document'], d['legal_document_label'], d['solution'], d['query'], d['document_type']] for d in t1_informations],
#              columns=['legal_document', 'legal_document_label', 'answer', 'query', 'document_type']).to_csv('t1_triple_only.csv', index=False)

In [ ]:
ts_rng = np.random.default_rng(12345)
ts_informations_permuted = ts_rng.permutation(ts_informations)
ts_rule = [d['rule'] for d in ts_informations_permuted]
# ts_queries = [d['query'] for d in ts_informations_permuted]

ts_generator = ts_qc()
ts_dataset = ts_generator.run(ts_informations_permuted)
ts = pd.DataFrame(ts_dataset, columns=['question', 'answer', 'ner', 'complete_sentence'])
ts['rule'] = ts_rule
# ts['query'] = ts_queries
ts['template_index'] = "TS"

del ts_rng
del ts_generator
# del ts_qc
del ts_dataset
del ts_informations
del ts_informations_permuted
del ts_rule
# del ts_queries
gc.collect()

print('TS Shape: ', ts.shape)
ts.head(5)

TS Shape:  (802, 6)


C:\Users\Rofif\anaconda3\envs\lexidvenv\lib\site-packages\ipykernel_launcher.py:161: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


,question,answer,ner,complete_sentence,rule,template_index
0,Dari apa Pembentukan Program Dapat Menjadi Bagian,Dari Pelaksanaan Program Pemerintah,"[B-QTY, O, B-SBJ, I-SBJ, B-MDL, B-ACT, B-OBJ]",Pembentukan Program Dapat Menjadi Bagian Dari ...,https://w3id.org/lex-id/data/Norm_Permen_LHK_2...,TS
1,Tentang apa Hakim Khusus Harus Menguasai Penge...,Tentang Perppu 2014 1 Pemilihan,"[B-QTY, O, B-SBJ, I-SBJ, B-MDL, B-ACT, B-OBJ]",Hakim Khusus Harus Menguasai Pengetahuan Tenta...,https://w3id.org/lex-id/data/Norm_Perppu_2014_...,TS
2,Haruskah Dokumen Pelaksanaan Anggaran Perubaha...,Ya,"[B-MDL, B-SBJ, I-SBJ, I-SBJ, I-SBJ, B-ACT, B-O...",Dokumen Pelaksanaan Anggaran Perubahan Harus M...,https://w3id.org/lex-id/data/Norm_Permen_Agama...,TS
3,Dari apa Permen Tan 2016 49 Kelompok Peternak ...,Dari Dinas,"[B-QTY, O, B-SBJ, I-SBJ, I-SBJ, I-SBJ, I-SBJ, ...",Permen Tan 2016 49 Kelompok Peternak Harus Men...,https://w3id.org/lex-id/data/Norm_Permen_Tan_2...,TS
4,"Beserta Muatannya, apakah Dimensi Kendaraan Pa...",Ya,"[B-QTY, B-QVA, O, B-SBJ, I-SBJ, B-MDL, I-MDL, ...",Dimensi Kendaraan Paling Sedikit Diketahui Uku...,https://w3id.org/lex-id/data/Norm_Permen_Hub_2...,TS


In [ ]:
ts.to_csv('semantic/dataset/dataset_semantic.csv', index=False)

#### TS Sample Question

In [ ]:
ts = pd.read_csv('semantic/dataset/dataset_semantic.csv')
list(ts.iloc[:50, 0])

['Dari apa Pembentukan Program Dapat Menjadi Bagian',
 'Tentang apa Hakim Khusus Harus Menguasai Pengetahuan',
 'Haruskah Dokumen Pelaksanaan Anggaran Perubahan Mendapatkan Persetujuan Dari Permen Agama 2019 9 Direktur Jenderal',
 'Dari apa Permen Tan 2016 49 Kelompok Peternak Harus Mendapat Rekomendasi Kelompok Peternak',
 'Beserta Muatannya, apakah Dimensi Kendaraan Paling Sedikit Diketahui Ukuran Tinggi Kendaraan',
 'Dalam Penjatuhan Putusan, apakah Surat Pernyataan Perdamaian Dapat Dijadikan pertimbangan KKEP',
 'Apa yang Harus Pengedar Benih Bina Memperoleh Dari UPTD',
 'Di Rumah Sakit Tujuan Penempatan, Hanya Dapatkah Peserta Wajib Kerja Dokter Spesialis Mandiri Menjalankan Praktik',
 'Dapatkah Tim Melibatkan Tenaga Profesional Ahli Sesuai Dengan Keperluan',
 'Melalui apa Saksi Dapat Mengikuti Seluruh Proses Pemungutan Suara',
 'Kepada apa Panitia Pembubaran Wajib Menyampaikan Pertanggungjawaban Pembubaran Perusda Bpr',
 'Apakah Pemeriksaan Wajib Dibuatkan Berita Acara Pemeriksaa

In [ ]:
del ts
gc.collect()

0